In [20]:
import requests
from bs4 import BeautifulSoup
import re
from jproperties import Properties
import json
import pandas as pd

In [2]:
configs = Properties()
with open('application.properties', 'rb') as config_file:
    configs.load(config_file)

In [3]:
dotabuff_filters = configs.get("dotabuff_filters").data
json_file_path = configs.get("json_file_path").data

In [4]:
url1 = "https://www.dotabuff.com/heroes"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

req = requests.get(url1, headers=headers)
soup = BeautifulSoup(req.content, "html.parser")
hero_links = soup.find_all("a", href=re.compile(r"/heroes/.*"))
heroes=[]
for link in hero_links:
    if link.find("div", class_="hero"):
        heroes.append({"hero_name": link.text.strip(), "link_name": link['href'][8:]})

In [5]:
heroes[1]

{'hero_name': 'Alchemist', 'link_name': 'alchemist'}

In [13]:
jsonstr = json.dumps(heroes_counter_data)

In [14]:
with open(json_file_path, "w") as f:
    f.write(jsonstr)

In [88]:
data_dict_dis={}
data_dict_wr={}
for hero in heroes[0:2]:
    single_hero_counter_data=[]
    url2 = f"https://www.dotabuff.com/heroes/{hero['link_name']}/counters{dotabuff_filters}"
    req = requests.get(url2, headers=headers)
    soup = BeautifulSoup(req.content, "html.parser")
    header = soup.find("header", string="Matchups")
    table = header.find_next_sibling("article").find("table")
    for row in table.find_all("tr"):
        counter_name_tag = row.find("a", class_="link-type-hero")
        if row.find("td") and hero_name_tag:
            disadvantage = row.find_all("td")[2]['data-value']
            wr = row.find_all("td")[3]['data-value']
            data_dict_dis.setdefault(counter_name_tag.text.strip(), {})[hero['hero_name']] = float(disadvantage)
            data_dict_wr.setdefault(counter_name_tag.text.strip(), {})[hero['hero_name']] = float(wr)
    print(hero['hero_name'])

Abaddon
Alchemist


In [93]:
df = pd.DataFrame.from_dict(data_dict_dis, orient='index')

df.fillna(0)

,Abaddon,Alchemist
Slark,2.8180,1.7394
Dark Seer,2.7863,0.2161
Undying,2.4031,-0.4823
Anti-Mage,2.3034,0.6232
Shadow Demon,2.2261,-0.2402
...,...,...
Arc Warden,-3.2389,-0.8156
Batrider,-3.3046,0.5728
Silencer,-3.4821,-1.1903
Spectre,-3.8734,-3.1863


In [91]:
df_sum = df['Abaddon'].fillna(0) + df['Alchemist'].fillna(0)

In [92]:
df_sum

Slark           4.5574
Dark Seer       3.0024
Undying         1.9208
Anti-Mage       2.9266
Shadow Demon    1.9859
                 ...  
Arc Warden     -4.0545
Batrider       -2.7318
Silencer       -4.6724
Spectre        -7.0597
Abaddon        -1.4208
Length: 123, dtype: float64

In [83]:
json_str = df_sum.to_json()

In [84]:
json_str

'{"Slark":4.5574,"Dark Seer":3.0024,"Undying":1.9208,"Anti-Mage":2.9266,"Shadow Demon":1.9859,"Elder Titan":1.0937,"Outworld Destroyer":2.8315,"Monkey King":2.8784,"Ancient Apparition":3.2812,"Meepo":-0.0869,"Shadow Fiend":2.4945,"Sniper":2.9704,"Terrorblade":2.4964,"Necrophos":2.3798,"Lina":1.5258,"Alchemist":0.0,"Earthshaker":0.3722,"Marci":1.3441,"Pangolier":1.3049,"Kunkka":0.5823,"Lone Druid":-0.8487,"Nature\'s Prophet":0.0792,"Lich":1.6072,"Lycan":-1.8145,"Io":-0.7872,"Phantom Lancer":-0.9509,"Weaver":1.0458,"Medusa":-0.5446,"Grimstroke":1.4075,"Bristleback":1.7422,"Keeper of the Light":0.1427,"Drow Ranger":1.1541,"Chen":-0.951,"Rubick":0.5215,"Sven":0.8416,"Viper":1.0679,"Troll Warlord":0.848,"Enchantress":1.337,"Underlord":-0.344,"Death Prophet":0.6792,"Riki":-0.5244,"Tusk":1.1841,"Lion":0.6061,"Leshrac":1.7109,"Dawnbreaker":0.572,"Bloodseeker":1.2805,"Chaos Knight":0.6967,"Lifestealer":1.8666,"Huskar":2.7137,"Primal Beast":0.8444,"Broodmother":-2.7459,"Tiny":-0.862,"Visage":-2.